# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.8075078261961934

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
7.9e-6 s/step ...done
MCMC, adapting ϵ (25 steps)
1.2e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
1.0e-5 s/step ...done
MCMC, adapting ϵ (100 steps)
8.3e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
6.8e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
6.8e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
9.6e-6 s/step ...done
MCMC (1000 steps)
6.8e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.7890217240979678], -6.4898469273717225, 1, DynamicHMC.AdjacentTurn, 0.7978422850088526, 3), NUTS_Transition{Array{Float64,1},Float64}([1.6995487779646523], -4.640188950581506, 1, DynamicHMC.DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.003424497605984067], -3.9276905351623363, 2, DynamicHMC.DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.003424497605984067], -4.595495685300759, 1, DynamicHMC.DoubledTurn, 0.5219140940669984, 1), NUTS_Transition{Array{Float64,1},Float64}([0.3855982652026098], -3.7850173923394825, 2, DynamicHMC.DoubledTurn, 0.8841316144303697, 3), NUTS_Transition{Array{Float64,1},Float64}([1.51691313530551], -4.2619862888092905, 2, DynamicHMC.DoubledTurn, 0.7743000050362602, 3), NUTS_Transition{Array{Float64,1},Float64}([0.6498927128360852], -3.6801173614861744, 1, DynamicHMC.AdjacentTurn, 0.9917188180623412, 3), NUTS_Transition{Array{Float64,1

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

405.04947300508036

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.1 0.86 0.97 1.0 1.0
  termination: AdjacentTurn => 30% DoubledTurn => 70%
  depth: 1 => 63% 2 => 37%


check the mean

In [11]:
mean(posterior_α)

0.6300391272485314

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*